In [1]:
import os
import sys
from pathlib import Path
import pandas as pd

%matplotlib inline

In [2]:
sys.path.append(r"../LUCinSA_helpers")
from rf import *

In [3]:
'''
PARAMETERS: modify in notebook_params notebook, then run that notebook and this cell to update here
DO NOT modify this cell
'''

%store -r basic_config
print("Basic Parameters: \n brdf_dir = {} \n gridCell = {} \n index_dir = {} \n home_dir = {}"
      .format(basic_config['brdf_dir'],basic_config['grid_cell'],basic_config['index_dir'],basic_config['home_dir']))
%store -r classification_params
print("Classification_Params: \n model_type = {} \n samp_pt_file = {} \n pix_vars = {} \n pixdf = {} \n model_name = {} \n lc_class = {} \n ranhold = {} \n impmeth = {}".format(classification_params['model_type'],classification_params['samp_pts'],classification_params['pix_vars'],classification_params['pixdf'],classification_params['model_name'],
classification_params['lc_mod'],classification_params['ranhold'],classification_params['impmeth']))

Basic Parameters: 
 brdf_dir = /home/sandbox-cel/paraguay_lc/stac/grid/003035/brdf 
 gridCell = 3035 
 index_dir = /home/downspout-cel/paraguay_lc/stac/grids/003035/brdf_ts/ms/evi2 
 home_dir = ~/data
Classification_Params: 
 model_type = RF 
 samp_pt_file = /home/downspout-cel/paraguay_lc/vector/sampleData/SamplePts_Nov2023_ALL.csv 
 pix_vars = /home/downspout-cel/paraguay_lc/vector/ptsgdb_Nov15.csv 
 pixdf = /home/downspout-cel/paraguay_lc/classification/RF/pixdf_lessSoy_Nov.csv 
 model_name = test0Nov 
 lc_class = All 
 ranhold = 29 
 impmeth = Impurity


In [4]:
lut=pd.read_csv('../Class_LUT.csv')
lut.drop(['Description'], axis=1, inplace=True)
print(lut.sort_values('LC_UNQ'))

    LC_UNQ               USE_NAME          Segmentation  LCTrans  \
0        1                  NoVeg                 NoVeg        1   
1        2             NoVeg_Bare                   NaN        2   
2        3            NoVeg_Built                   NaN        3   
3        7            NoVeg_Water                   NaN        7   
4        9             Mixed-path                   NaN        9   
5       10                Cleared                   NaN       10   
6       11          TreePlant-new                   NaN       11   
7       12      Grassland-Natural                   NaN       15   
8       13      Grassland-Planted  LowVeg_grass_managed       15   
9       15              Grassland          LowVeg_grass       15   
10      17  Grassland-Natural-wet                   NaN       15   
11      18        Mixed-GrassEdge                   NaN       19   
12      19        Mixed-FieldEdge                   NaN       19   
13      20                 LowVeg               

## start with default models with pixel-only data:

In [5]:
samp_pts = pd.read_csv(classification_params['samp_pts'])
pix_vars = pd.read_csv(classification_params['pix_vars'])

pix_data = pix_vars.merge(samp_pts, left_on='OID_', right_on='OID_', how='left')

pix_data.drop(['LC2'], axis=1, inplace=True)
pixdf = pix_data.merge(lut, left_on='Class', right_on='USE_NAME', how='left')
print('sample breakdown by LC17 class:')
print(pixdf['LC17_name'].value_counts())

print('default rf model with all sample pixels and pixel only data')

sample breakdown by LC17 class:
Crops-Soybeans       4443
Mixed-path           3644
Crops-mix            2870
unknown              1645
Mixed-VegEdge        1626
Grassland-Planted     981
Trees-Forest          819
Trees-disturbed       788
TreePlant             743
Crops-Yerba-Mate      709
Grassland-Natural     517
Crops-Rice            496
Shrub                 297
Crops-Sugar           292
NoVeg_Water           264
Crops-Orchard         254
NoVeg_Bare            235
Crops-Corn            231
NoVeg_Built           224
Grass_tree-mix         90
NewPlant               38
Crops-Vineyard         12
Name: LC17_name, dtype: int64
default rf model with all sample pixels and pixel only data


In [6]:
dropClass = ['Crops-Vineyard','NewPlant']
pixdf = pixdf[~pixdf['LC17_name'].isin(dropClass)]
print(pixdf['LC17_name'].value_counts())

Crops-Soybeans       4443
Mixed-path           3644
Crops-mix            2870
unknown              1645
Mixed-VegEdge        1626
Grassland-Planted     981
Trees-Forest          819
Trees-disturbed       788
TreePlant             743
Crops-Yerba-Mate      709
Grassland-Natural     517
Crops-Rice            496
Shrub                 297
Crops-Sugar           292
NoVeg_Water           264
Crops-Orchard         254
NoVeg_Bare            235
Crops-Corn            231
NoVeg_Built           224
Grass_tree-mix         90
Name: LC17_name, dtype: int64


## prep pixel datasets by reducing sample
### First by sampling method (reducing dominant CAN soy pts)
### can also do by field size, etc.

In [ ]:
#### model_name = pixdf_25Soy
##   removes 3/4 of the soy points because they are far overrepresented
#pixdf_25Soy = pixdf[(pixdf['rand']>.9) | (pixdf['SampMethod'] != 'CAN - unverified in GE')]
#print('25Soy sample breakdown by LC17 class:')
#print(pixdf_25Soy['LC17_name'].value_counts())
#pixdf_path = os.path.join(classification_params['model_dir'],'pixdf_25Soy.csv')
#pd.DataFrame.to_csv(pixdf_25Soy, pixdf_path)

In [7]:
#### model_name = pixdf_lessSoy
# removes all soy points that sre not verified in Google Earth
pixdf_lessSoy = pixdf[pixdf['SampMethod'] != 'CAN - unverified in GE']
print('there are now {} pts in the training set after dropping CAN soy'.format(len(pixdf_lessSoy)))
print('LessSoy sample breakdown by LC17 class:')
print(pixdf_lessSoy['LC17_name'].value_counts())
pixdf_path = os.path.join(classification_params['model_dir'],'pixdf_lessSoy_Nov.csv')
pd.DataFrame.to_csv(pixdf_lessSoy, pixdf_path)

there are now 17092 pts in the training set after dropping CAN soy
LessSoy sample breakdown by LC17 class:
Mixed-path           3644
Crops-mix            2870
unknown              1645
Mixed-VegEdge        1626
Grassland-Planted     981
Trees-Forest          819
Trees-disturbed       788
TreePlant             743
Crops-Yerba-Mate      709
Grassland-Natural     517
Crops-Rice            496
Crops-Soybeans        325
Shrub                 297
Crops-Sugar           292
NoVeg_Water           264
Crops-Orchard         254
NoVeg_Bare            235
Crops-Corn            231
NoVeg_Built           224
Grass_tree-mix         90
Name: LC17_name, dtype: int64


## if polygons are available, can combine pixel and polygon dfs and create rf datasets for points with polygons and those without:

In [ ]:
poly_data = pd.read_csv(classification_params['samp_poly'])
#rename column names that also occur in pixel df
poly_data.rename(columns={'area':'areaSeg'}, inplace=True)
all_data = pixdf.merge(poly_data, left_on='OID_', right_on='OID_', how='left')
polypixdf_path = os.path.join(classification_params['model_dir'],'pts_polyData_joinCheck.csv')
pd.DataFrame.to_csv(all_data, polypixdf_path, sep=',', na_rep='NaN', index=True)

### first create dataset for points outside of polygons (here we have no variables to add to the original model)

In [ ]:
outsideSeg = all_data[all_data['areaSeg'].isna()]
print(f'of the {all_data.shape[0]} sample points in our dataset, {outsideSeg.shape[0]} are outside of our segmented polygons')
print(outsideSeg['LC17_name'].value_counts())
out_dir = os.path.join(classification_params['model_dir'],'outside')
rfout17 = rf_model(outsideSeg,out_dir,'All','Permutation',29,'Fullsamp')

And for model with more balanced soy representation (25Soy):

In [ ]:
all_data_25Soy = pixdf_25Soy.merge(poly_data, left_on='OID_', right_on='OID_', how='left')
outsideSeg_25Soy = all_data_25Soy[all_data_25Soy['areaSeg'].isna()]
print(outsideSeg_25Soy['LC17_name'].value_counts())
out_dir = os.path.join(classification_params['model_dir'],'outside')
rfout17_lessSoy = rf_model(outsideSeg_lessSoy,out_dir,'All','Permutation',29,'LessSoy')

#### now create dataset for points inside of polygons (here we want to add some variables first)

In [ ]:
#poly_data['AvgU'] = poly_data.apply(lambda x:count([x[c] for c in df.columns if c.endswith('U')]),axis=1)
#TODO: calculate these in pandas as above
poly_data.rename(columns={'areaSeg':'var_areaSeg','AVGU':'var_AVGU','AVGR':'var_AVGR','AVGSTD':'var_AVGSTD','MAXR':'var_MAXR','MINR':'var_MINR','STDU':'var_STDU','MINU':'var_MINU','MAXU':'var_MAXU','rU':'var_RU'}, inplace=True)
polyvars = pixdf.merge(poly_data, left_on='OID_', right_on='OID_', how='left')
withinSeg = polyvars[polyvars['var_areaSeg'] > 0]

print(withinSeg['LC17_name'].value_counts())
out_dir = os.path.join(classification_params['model_dir'],'within')
#rfin17 = rf_model...

In [ ]:
all_data_lessSoy = pixdf_lessSoy.merge(poly_data, left_on='OID_', right_on='OID_', how='left')
withinSeg_lessSoy = all_data_lessSoy[all_data_lessSoy['var_areaSeg']>0]
print(withinSeg_lessSoy['LC17_name'].value_counts())
out_dir = os.path.join(classification_params['model_dir'],'within')
#rfin17_lessSoy = rf_model(...

## To save an html copy of this notebook with all outputs:
(these two cells should be last in notebook)

In [ ]:
%%script echo skipping
### comment out above line and run this cell to print output as html

out_name = str(basic_config['country']+'6a_RandomFoest_VariableDataframe'+'_model'+str(classification_params['model_name'])+'_'+'Tests1')
!jupyter nbconvert --output-dir='./Outputs' --to html --no-input --ExecutePreprocessor.store_widget_state=True --output=$out_name 6b_RandomFoest_ModelComparisons.ipynb